# **Underwriting Analysis - Credit Denials**

## Combined

In [6]:
# Current Analysis
from ipyfilechooser import FileChooser
from IPython.display import clear_output, display, HTML
from ipywidgets import Output
import pandas as pd
from scipy.stats import fisher_exact
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from tqdm.notebook import tqdm

# Create a FileChooser widget
fc = FileChooser()

# Create an output widget
output = Output()

# Define a function that will be called when a file is selected
def on_file_selected(filechooser):
    # Clear the output widget
    output.clear_output(wait=True)
    
    file_name = filechooser.selected
    
    # Formatting of the results section
    BOLD = '<b>'
    UNDERLINE = '<u>'
    ITALIC = '<i>'
    END = '</b></u></i>'

    # This pulls in the data from the SQL query saved in CSV format
    underwriting = pd.read_csv(file_name)

    # This sets the "LoanType" to run the analysis
    loan_types = ['Conventional', 'FHA', 'VA', 'FmHA']

    def apply_filters(analysis, purpose, loancount, show_all, print_list):
        display(HTML("<br><br>"))
        for loan_type in tqdm(loan_types):
            display(HTML("<br><br>"))
            
            underwriting_loan_type = underwriting[underwriting['LoanType'] == loan_type]
            underwriting_loan_type = underwriting_loan_type[(underwriting_loan_type['HmdaActionTaken'] == "Loan Originated") | (underwriting_loan_type['HmdaActionTaken'] == "Application denied") | (underwriting_loan_type['HmdaActionTaken'] == "Application approved but not accepted")]
            underwriting_loan_type.loc[underwriting_loan_type['HmdaActionTaken'] == "Application approved but not accepted", 'HmdaActionTaken'] = "Loan Originated"
            underwriting_loan_type = underwriting_loan_type[(underwriting_loan_type['Purpose'] == "Purchase") | (underwriting_loan_type['Purpose'] == "Refinance")]
            underwriting_loan_type = underwriting_loan_type[underwriting_loan_type['ApplicationDate'] != "NULL"]
            
# RACE ANALYSIS
            if analysis == "Race" or analysis == 'All':
            
                if purpose != "Combined":
                    underwriting_loan_type = underwriting_loan_type[underwriting_loan_type['Purpose'] == purpose]

                # This is where the comparisons are performed
                asian = underwriting_loan_type[(underwriting_loan_type['Race'] == "White") | (underwriting_loan_type['Race'] == "Asian")]
                black = underwriting_loan_type[(underwriting_loan_type['Race'] == "White") | (underwriting_loan_type['Race'] == "Black or African American")]
                indian = underwriting_loan_type[(underwriting_loan_type['Race'] == "White") | (underwriting_loan_type['Race'] == "American Alaska or Indian")]
                islander = underwriting_loan_type[(underwriting_loan_type['Race'] == "White") | (underwriting_loan_type['Race'] == "Native Hawaiian or Other Pacific Islander")]

                def print_results(race, race_name):
                    race_crosstab = pd.crosstab(race.Race, race.HmdaActionTaken)
                    #display(HTML("<br>"))
                    
                    if race_name in race_crosstab.index:
                        if (race_crosstab.loc[:, 'Application denied'] >= loancount).all() and (race_crosstab.loc[:, 'Loan Originated'] >= loancount).all():
                            results = fisher_exact(race_crosstab, alternative="greater")
                            if results[1] <= 0.05 and results[0] >= 1.2:
                                display(HTML(BOLD + race_name + ' (' + loan_type + ')' + END))
                                display(HTML("<br>"))
                                display(race_crosstab)
                                display(HTML("<br>"))
                                display(HTML("Odds Ratio: "  + BOLD + str(round(results[0], 1)) + END + "<br>" + "p-value: "  + BOLD + str(round(results[1], 4)) + END))
                                display(HTML(BOLD + ITALIC + "Statistically significant denial rate" + END))
                                display(HTML("<br><br>"))

                                # Print the list of denied Target loans
                                if print_list == 'Include ':
                                    display(HTML("<br>"))
                                    display(HTML(BOLD + UNDERLINE + "List of denied loans:" + END))
                                    display(HTML("<br>"))
                                    denied_loans = race[(race["HmdaActionTaken"] == "Application denied") & (race["Race"] != "White")][["LoanId", "Race", "HmdaActionTaken", "Purpose"]]
                                    html_table = denied_loans.to_html(index=False)
                                    # Add inline CSS to adjust the font size
                                    html_table = '<style> table {font-size: 0.9em;} </style>' + html_table
                                    display(HTML(html_table))
                                    display(HTML("<br><br>"))
                    
                            elif show_all =='All':
                                display(HTML(BOLD + race_name + ' (' + loan_type + ')' + END))
                                display(HTML("<br>"))
                                display(race_crosstab)
                                display(HTML("<br>"))
                                display(HTML(BOLD +"Fisher's test is not statistically significant for " + race_name + END))
                                display(HTML("<br><br>"))
                        
                        elif show_all =='All':
                            display(HTML(BOLD + race_name + ' (' + loan_type + ')' + END))
                            display(HTML("<br>"))
                            display(race_crosstab)
                            display(HTML("<br>"))
                            display(HTML(BOLD + "Not enough data to perform Fisher's test for " + race_name + END))
                            display(HTML("<br><br>"))
                    
                    elif show_all =='All':
                        display(HTML(BOLD + race_name + ' (' + loan_type + ')' + END))
                        display(HTML("<br>"))
                        display(race_crosstab)
                        display(HTML("<br>"))
                        display(HTML(BOLD + "No data available for " + race_name + END))
                        display(HTML("<br><br>"))

                # This presents the results for Black or African American
                print_results(black, 'Black or African American')

                # This presents the results for American Alaska or Indian
                print_results(indian, 'American Alaska or Indian')

                # This presents the results for Native Hawaiian or Other Pacific Islander
                print_results(islander, 'Native Hawaiian or Other Pacific Islander')

                # This presents the results for Asian
                print_results(asian, 'Asian')

    # This sets the "Hispanic" and "Comparison" groups for ethnicity and sex analysis
            hispanic = underwriting_loan_type[(underwriting_loan_type['Class'] == 'NonHispanic') | (underwriting_loan_type['Class'] == 'Hispanic')]
            malefemale = underwriting_loan_type[(underwriting_loan_type['Sex'] == 'Male') | (underwriting_loan_type['Sex'] == 'Female')]

# ETHNICITY RESULTS
            if analysis == 'All' or analysis == 'Ethnicity':
                hispanic_crosstab = pd.crosstab(hispanic['Class'], hispanic['HmdaActionTaken'])
                if (hispanic_crosstab.loc[:, 'Loan Originated'] >= loancount).all() and (hispanic_crosstab.loc[:, 'Application denied'] >= loancount).all():
                    results = fisher_exact(hispanic_crosstab, alternative='greater')
                    if results[1] <= 0.05 and results[0] >= 1.2:
                        display(HTML(BOLD + f'Ethnicity ({loan_type})' + END))
                        display(HTML("<br>"))
                        display(HTML(hispanic_crosstab.to_html()))
                        display(HTML("<br>"))
                        display(HTML("Odds Ratio: " + BOLD + str(round(results[0], 2)) + END + "<br>" + "p-value: " + BOLD + str(round(results[1], 4)) + END))
                        display(HTML(BOLD + ITALIC + "Statistically significant denial rate" + END))
                        display(HTML("<br>"))
                        
                        # Print the list of denied Hispanic loans
                        if print_list == 'Include':
                            display(HTML(BOLD + UNDERLINE + "List of denied loans:" + END))
                            denied_loans_class = underwriting_loan_type[(underwriting_loan_type["HmdaActionTaken"] == "Application denied") & (underwriting_loan_type["Class"] != "NonHispanic") & (underwriting_loan_type["Class"] != "EXCLUDED")]
                            df = denied_loans_class[["LoanId", "Class", "HmdaActionTaken", "Purpose"]]
                            # Use to_html() to format the DataFrame as an HTML table and exclude the index
                            html_table = df.to_html(index=False)
                            
                            # Add inline CSS to adjust the font size
                            html_table = '<style> table {font-size: 0.9em;} </style>' + html_table
                            display(HTML(html_table))
                            display(HTML("<br>"))
                    
                    elif show_all =='All':
                        display(HTML(BOLD + f'Ethnicity for {loan_type} Loans' + END))
                        display(HTML("<br>"))
                        display(HTML(hispanic_crosstab.to_html()))
                        display(HTML("<br>"))
                        display(HTML(BOLD + "Fisher's test is not statistically significant for Ethnicity" + END))
                        display(HTML("<br>"))
                        
                elif show_all == 'All':
                    display(HTML(BOLD + f'Ethnicity for {loan_type} Loans' + END))
                    display(HTML("<br>"))
                    display(HTML(hispanic_crosstab.to_html()))
                    display(HTML("<br>"))
                    display(HTML(BOLD + "Not all groups have at least 15 originated and 15 denied loans" + END))
                    display(HTML("<br>"))
                    
            elif analysis == 'Sex' or analysis == 'Age' or analysis == 'Race':
                pass        

# SEX RESULTS
            if analysis == 'All' or analysis == 'Sex':
                sex_crosstab = pd.crosstab(malefemale['Sex'], malefemale['HmdaActionTaken'])
                if (sex_crosstab.loc[:, 'Loan Originated'] >= loancount).all() and (sex_crosstab.loc[:, 'Application denied'] >= loancount).all():
                    results = fisher_exact(sex_crosstab, alternative='greater')
                    if results[1] <= 0.05 and results[0] >= 1.2:
                        display(HTML(BOLD + f'Sex ({loan_type})' + END))
                        display(HTML("<br>"))
                        display(HTML(sex_crosstab.to_html()))
                        display(HTML("<br>"))
                        display(HTML("Odds Ratio: " + BOLD + str(round(results[0], 2)) + END + "<br>" + "p-value: " + BOLD + str(round(results[1], 4)) + END))
                        display(HTML(BOLD + ITALIC + "Statistically significant denial rate" + END))
                        display(HTML("<br>"))
                        
                        # Print the list of denied Hispanic loans
                        if print_list == 'Include':
                            display(HTML(BOLD + UNDERLINE + "List of denied loans:" + END))
                            denied_loans_sex = underwriting_loan_type[(underwriting_loan_type["HmdaActionTaken"] == "Application denied") & (underwriting_loan_type["Sex"] != "Male")]
                            df = denied_loans_sex[["LoanId", "Sex", "HmdaActionTaken", "Purpose"]]
                            
                            # Use to_html() to format the DataFrame as an HTML table and exclude the index
                            html_table = df.to_html(index=False)
                            
                            # Add inline CSS to adjust the font size
                            html_table = '<style> table {font-size: 0.9em;} </style>' + html_table
                            display(HTML(html_table))
                            display(HTML("<br>"))
                            
                    elif show_all =='All':
                        display(HTML(BOLD + f'Sex ({loan_type})' + END))
                        display(HTML("<br>"))
                        display(HTML(sex_crosstab.to_html()))
                        display(HTML("<br>"))
                        display(HTML(BOLD + "Fisher's test is not statistically significant for Gender" + END))
                        display(HTML("<br>"))
                        
                elif show_all =='All':
                    display(HTML(BOLD + f'Sex ({loan_type})' + END))
                    display(HTML("<br>"))
                    display(HTML(sex_crosstab.to_html()))
                    display(HTML("<br>"))
                    display(HTML(BOLD + "Not all groups have at least 15 originated and 15 denied loans" + END))
                    display(HTML("<br>"))
                    
            elif analysis == 'Ethnicity' or analysis == 'Age' or analysis == 'Race':
                pass

# AGE RESULTS 
            if analysis == 'All' or analysis == 'Age':
                age_crosstab = pd.crosstab(underwriting_loan_type['Age'], underwriting_loan_type['HmdaActionTaken'])
                if (age_crosstab.loc[:, 'Loan Originated'] >= loancount).all() and (age_crosstab.loc[:, 'Application denied'] >= loancount).all():
                    results = fisher_exact(age_crosstab, alternative='greater')
                    if results[1] <= 0.05 and results[0] >= 1.2:
                        display(HTML(BOLD + f'Age ({loan_type})' + END))
                        display(HTML("<br>"))
                        display(HTML(age_crosstab.to_html()))
                        display(HTML("<br>"))
                        display(HTML("Odds Ratio: " + BOLD + str(round(results[0], 2)) + END + "<br>" + "p-value: " + BOLD + str(round(results[1], 4)) + END))
                        display(HTML(BOLD + ITALIC + "Statistically significant denial rate" + END))
                        display(HTML("<br>"))

                        # Print the list of denied Hispanic loans
                        if print_list == 'Include':
                            display(HTML(BOLD + UNDERLINE + "List of denied loans:" + END))
                            denied_loans_age = underwriting_loan_type[(underwriting_loan_type["HmdaActionTaken"] == "Application denied") & (underwriting_loan_type["Age"] != "Under 62 Yrs Old")]
                            df = denied_loans_age[["LoanId", "Age", "HmdaActionTaken", "Purpose"]]
                            
                            # Use to_html() to format the DataFrame as an HTML table and exclude the index
                            html_table = df.to_html(index=False)
                            
                            # Add inline CSS to adjust the font size
                            html_table = '<style> table {font-size: 0.9em;} </style>' + html_table
                            display(HTML(html_table))
                            display(HTML("<br>"))
                            
                    elif show_all =='All':
                        display(HTML(BOLD + f'Age for {loan_type} Loans' + END))
                        display(HTML("<br>"))
                        display(HTML(age_crosstab.to_html()))
                        display(HTML("<br>"))
                        display(HTML(BOLD + "Fisher's test is not statistically significant for Age" + END))
                        display(HTML("<br>"))
                        
                elif show_all =='All':
                    display(HTML(BOLD + f'Age for {loan_type} Loans' + END))
                    display(HTML("<br>"))
                    display(HTML(age_crosstab.to_html()))
                    display(HTML("<br>"))
                    display(HTML(BOLD + "Not all groups have at least 15 originated and 15 denied loans" + END))
                    display(HTML("<br>"))
                    
            elif analysis == 'Ethnicity' or analysis == 'Sex' or analysis == 'Race':
                pass

    # Create a radio button widget with options for statistically significant results
    analysis_widget = widgets.ToggleButtons(
        options=["All", "Race", "Ethnicity", "Age", "Sex"],
        description='Demographic Analysis:',
        disabled=False,
        button_style='', # 'success', 'info', 'warning', 'danger' or ''
        tooltips=['All (Default)', 'Race', 'Ethnicity', 'Age', 'Sex'],
    ) 

    # Create a radio button widget with options for 'Purpose'
    purpose_widget = widgets.ToggleButtons(
        options=["Combined", "Purchase", "Refinance"],
        description='Purpose:',
        disabled=False,
        button_style='', # 'success', 'info', 'warning', 'danger' or ''
        tooltips=['Filter by Purchase', 'Filter by Refinance', 'Include both Purchase and Refinance'],
    )
    
    # Create a radio button widget with options for 15 or 30 Loan requirement
    loancount_widget = widgets.ToggleButtons(
        options=[15, 30],
        description='Loan Count Requirement:',
        disabled=False,
        button_style='', # 'success', 'info', 'warning', 'danger' or ''
        tooltips=['Semiannual', 'Annual'],
    )

    # Create a radio button widget with options for print list of loans
    show_results_widget = widgets.ToggleButtons(
        options=["Stat. Significant", "All"],
        description='Show Results:',
        disabled=False,
        button_style='', # 'success', 'info', 'warning', 'danger' or ''
        tooltips=['Exclude the list of loans (Default)', 'Include the list of loans'],
    )

    # Create a radio button widget with options for print list of loans
    print_list_widget = widgets.ToggleButtons(
        options=["Exclude ", "Include "],
        description='Show List of Loans:',
        disabled=False,
        button_style='', # 'success', 'info', 'warning', 'danger' or ''
        tooltips=['Exclude the list of loans (Default)', 'Include the list of loans'],
        icons=['times', 'list']
    )

    # Use the interact function to apply the selected filters to the code
    interact(apply_filters,analysis=analysis_widget, purpose=purpose_widget, loancount=loancount_widget, show_all=show_results_widget, print_list=print_list_widget);

# Set the function to be called when a file is selected
fc.register_callback(on_file_selected)

# Display the FileChooser widget
display(fc)
fc.title = '<b>Select Dataset for UW Analysis by Race</b>'

# Display the output widget
display(output)


FileChooser(path='C:\Users\colby\OneDrive\Documents\Data Analysis\Python_Project_Fair_Lending_Analysis\ipynb_f…

Output()

interactive(children=(ToggleButtons(description='Demographic Analysis:', options=('All', 'Race', 'Ethnicity', …

---